# How to use a SmartLLMChain

A SmartLLMChain is a form of self-critique chain that can help you if have particularly complex questions to answer. Instead of doing a single LLM pass, it instead performs these 3 steps:
1. Ideation: Pass the user prompt n times through the LLM to get n output proposals (called "ideas"), where n is a parameter you can set 
2. Critique: The LLM critiques all ideas to find possible flaws and picks the best one 
3. Resolve: The LLM tries to improve upon the best idea (as chosen in the critique step) and outputs it. This is then the final output.

SmartLLMChains are based on the SmartGPT workflow proposed in https://youtu.be/wVzuvf9D9BU.

Note that SmartLLMChains
- use more LLM passes (ie n+2 instead of just 1)
- only work then the underlying LLM has the capability for reflection, which smaller models often don't
- only work with underlying models that return exactly 1 output, not multiple

This notebook demonstrates how to use a SmartLLMChain.

##### Same LLM for all steps

In [11]:
from langchain.prompts import PromptTemplate
from langchain_experimental.smart_llm import SmartLLMChain

import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
# 使用 通义千问 代替openai
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",   # ✅ 用通义千问提供的模型名，比如 qwen-plus / qwen-max / qwen-turbo
    temperature=0
)


As example question, we will use "I have a 12 liter jug and a 6 liter jug. I want to measure 6 liters. How do I do it?". This is an example from the original SmartGPT video (https://youtu.be/wVzuvf9D9BU?t=384). While this seems like a very easy question, LLMs struggle do these kinds of questions that involve numbers and physical reasoning.

As we will see, all 3 initial ideas are completely wrong - even though we're using GPT4! Only when using self-reflection do we get a correct answer. 

In [12]:
hard_question = "我有一个12升的壶和一个6升的壶。我想量取6升。该怎么做？"

So, we first create an LLM and prompt template

In [4]:
prompt = PromptTemplate.from_template(hard_question)

Now we can create a SmartLLMChain

In [13]:
chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=3, verbose=False)

Now we can use the SmartLLM as a drop-in replacement for our LLM. E.g.:

In [14]:
chain.invoke({})

{'resolution': "Certainly! Let's work step by step to determine which of the three answer options the researcher would consider the **best**, then **improve it**, and finally present the **final improved answer in full**.\n\n---\n\n### Step 1: Determine the Best Answer\n\nWe analyze each of the three answer options based on clarity, structure, and efficiency:\n\n| Answer | Clarity | Structure | Efficiency | Notes |\n|--------|---------|-----------|------------|-------|\n| Idea 1 | ✅ Good | ✅ Logical flow, clear steps | ✅ Efficient | First and clear explanation |\n| Idea 2 | ✅ Good | ✅ Well-structured | ❌ Slightly wordy | Same logic, more verbose |\n| Idea 3 | ✅ Good | ✅ Clear steps | ❌ Repetitive | Same as the others |\n\n**Conclusion**: All are logically sound, but **Idea 1** is the most **concise**, **clear**, and **logically structured**. It introduces the problem, labels the jugs, and walks through the steps in a straightforward way.\n\n---\n\n### Step 2: Improve the Best Answer\n\

##### Different LLM for different steps

You can also use different LLMs for the different steps by passing `ideation_llm`, `critique_llm` and `resolve_llm`. You might want to do this to use a more creative (i.e., high-temperature) model for ideation and a more strict (i.e., low-temperature) model for critique and resolution.

In [15]:
chain = SmartLLMChain(
    ideation_llm=ChatOpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        model="qwen-plus",   # ✅ 用通义千问提供的模型名，比如 qwen-plus / qwen-max / qwen-turbo
        temperature=0.7
    ),
    llm=ChatOpenAI(
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        base_url="https://api.deepseek.com",
        model="deepseek-chat",
        temperature=0,
    ),  # will be used for critique and resolution as no specific llms are given
    prompt=prompt,
    n_ideas=3,
    verbose=True,
)

In [16]:
chain.invoke({})



> Entering new SmartLLMChain chain...
Prompt after formatting:
我有一个12升的壶和一个6升的壶。我想量取6升。该怎么做？
Idea 1:
当然可以！我们一步一步来解决这个问题。

---

### 问题重述：
你有一个 **12升的壶** 和一个 **6升的壶**，你的目标是**量出6升水**。

---

### 解法步骤：

1. **把12升的壶装满水**。  
   - 现在：12升壶有 **12升**，6升壶有 **0升**

2. **用12升壶往6升壶倒水，直到6升壶满**。  
   - 现在：12升壶剩下 **6升**，6升壶有 **6升**

---

### 结果：
现在，**12升的壶中正好剩下6升水**，任务完成！

---

### 总结：
你通过把12升壶的水倒入6升壶，得到了**正好6升**留在大壶中。

✅ **成功量出6升水！**
Idea 2:
好的，我们来一步一步地解决这个问题：

---

### **问题：**
你有一个 **12升的壶** 和一个 **6升的壶**，你想量出 **6升水**。该怎么做？

---

### **目标：**
利用这两个壶，最终在其中一个壶中**准确量出6升水**。

---

### **已知条件：**
- 12升壶是空的。
- 6升壶也是空的。
- 你可以**无限量地取水**（比如从水龙头或水井取水）。
- 你可以**倒空壶**或将水从一个壶倒入另一个壶。

---

### **解决步骤：**

我们来一步步操作：

#### 步骤 1：
将 **6升壶装满水**。

- 6升壶：6升  
- 12升壶：0升

#### 步骤 2：
将 **6升壶的水倒入12升壶**。

- 6升壶：0升  
- 12升壶：6升 ✅

---

### **结果：**
现在 **12升壶中有6升水**，你已经成功量出了6升水！

---

### ✅ 答案总结：
1. 把6升壶装满水。
2. 把6升壶的水倒入12升壶。
3. 现在12升壶中正好有6升水。

---

如果你有其他目标（比如量出其他升数），也可以继续用类似的方法一步步推理。需要我帮你吗？😊
Idea 3:
当然！这是一个经典的水壶问题。我们一步一步来解决这个问题。

--

{'resolution': '首先，将12升的壶装满水。  \n然后，将12升壶中的水倒入6升壶，直到6升壶满。  \n此时，12升壶中剩下6升水，任务完成。'}